# Maximum number sequence Test

# Test Description
This script will test the functionality of number of sequence, which means the number of sequence executed.

As per the datasheet maximum number of sequences is 32K.

To define one sequence, minimum 3 tasks are required (Start,seq,and end) 
but maximum task we can generate is 64K 
so we can test maximum 64K(65536)/3 = 21K(21845) sequence number is possible if unique tasks are defined for all sequence.

For Example seq_num=3 seq_loop=2
To test this  We need to generate total 10 Tasks
sequence1 created with Task1(sinewave),Task2(square) and Task3(Triangle)
sequence2 created with Task4(sinewave),Task5(square) and Task6(Triangle)
sequence3 created with Task7(sinewave),Task8(square) and Task9(Triangle)
Task10 is created DC wave.

 Also we use CPU trigger for each task to play so we require total (seq_num*seq_loop*3+1) = 19 triggers.
#And if we got DC wave on 19th Task, we can say that Test is passed.
 if we got DC wave for any other task, we can say that Test is failed. 


# Test Equipment
1. Tabor Proteus 9484M
2. Keysight Oscilloscope MSO9254A
3. 4 SMA to BNC Cable

# Hardware Connection
    Proteus Module    --->     Oscilloscope
    Channel 1         --->      Channel 1
    Channel 2         --->      Channel 2
    Channel 3         --->      Channel 3
    Channel 4         --->      Channel 4


# Test Procedure
1. Connect all the channels of Proteus module with the concerned channel of oscilloscope.
2. Run the script and observe output on scope.

Proteus release - Aurora

In [11]:
# Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
#matplotlib notebook
import pyvisa as visa
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy import signal
import math
import pdb

In [12]:
# Connect to instrument
connection = 'LAN' #input ("Do you wish to connect via LAN/PXI/LOCAL/USB ")
if connection == 'PXI':   
    # not setting this command means DLL are taken from SYS32
    #set_lib_dir_path(r'D:\Projects\ProteusAwg_Anabelle\x64\Debug') 
    # for service connection enter instrument IP
    # for PXI DLL connection enter the module slot number with Auto=False
    # when Auto=True the lowest module will be connected
    inst=connect("2",Auto=False)
    
elif connection == 'LAN':
    proteus_addr = 'TCPIP::192.90.70.22::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'USB':
    proteus_addr = 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
elif connection == 'LOCAL':
    proteus_addr = 'TCPIP::127.0.0.1::5025::SOCKET'
    try:
        inst = TEVisaInst(proteus_addr)
    except TEVisaInst.Error as  ex1:
        print('Couldn\'t connect to \'%s\', exiting now...' % proteus_addr)
        sys.exit()
        
else:
    print("Please select the conection method first")
# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print('Connected to: ' + resp)
inst.default_paranoia_level = 2


Connected to: Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 


In [13]:
# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))

# Get model dependant parameters:
if model_name.startswith('P948'):
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 1
elif model_name.startswith('P908'):
    bpp = 1
    max_dac = 255
    wpt_type = np.uint8
    offset_factor = 1
else:
    bpp = 2
    max_dac = 65535
    wpt_type = np.uint16
    offset_factor = 2
    
half_dac = max_dac / 2.0    
# Get the DAC mode (8 bits or 16 bits)

resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0': 
    dac_mode=16 
else: dac_mode=8
   
print('DAC {0} bits'.format(dac_mode))

Model: P9484M 
DAC 16 bits


In [14]:
# Several initializations ..
inst.send_scpi_cmd('*CLS; *RST')
#proteus.send_scpi_cmd(':INST:CHAN 2')
inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
resp = inst.send_scpi_query(':SYST:ERR?')
print(resp)
# Get number of channels
num_channels = inst.send_scpi_query(":INST:CHAN? MAX")
print("Number of channels: " + num_channels)
#num_channels = int(resp)

0, no error
Number of channels: 4


In [15]:
FREQ = 1000
inst.send_scpi_cmd(":FREQ:RAST {0}MHz".format(FREQ))
resp = inst.send_scpi_query(":FREQ:RAST?")
freq = float(resp)
print ("DAC Generate Freq:{0}".format(freq))

DAC Generate Freq:1000000000.0


In [16]:
scope_addr2=  'USB0::0x2A8D::0x900E::MY55490134::INSTR'

# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
    
    
    
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))

for i in range(4):
    scope.write(':CHANnel{}:DISPlay OFF'.format(i+1)) 
    scope.write(':MEASure:CLEar')


*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


In [ ]:
channel_select = [1,2,3,4]
for in_channelselect in range(len(channel_select)):
    # Select Channel & Clear it for new operation
    print('Test start for channel no. {}'.format(channel_select[in_channelselect]))
    
    scope.write('*CLS;:DISPlay:CGRade:LEVels ')
    scope.write(':CHANnel{}:DISPlay ON'.format(channel_select[in_channelselect]))
    
    resp = inst.send_scpi_query(':INST:CHAN?')

    
    #     Several initializations ..
    inst.send_scpi_cmd('*CLS; *RST')
    inst.send_scpi_cmd(':TRAC:DEL:ALL') # Delete all segments of the programmable channel's DDR.
        
      
        
    # Build sine wave of 2 cycle for Segment 1
    segnum = 1
    seglen = 1024
    ncycles1 = 2
    cyclelen = (seglen/ncycles1)
    waves = [ None for _ in range(3)]
    # sin wave:
    x = np.linspace(start=0, stop=2 * np.pi * ncycles1, num=seglen, endpoint=False)
    y = (np.sin(x) + 1.0) * half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    waves[0] = y.astype(wpt_type)
#     plt.plot(x,y)
#     print("************* WAVEFORM*****************")
    # Download sine wave to channel 1
    wav = waves[0]
#     print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))    
    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
    # Select the segment
    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000           
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}" after writing binary values'.format(resp))
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
    inst.send_scpi_cmd(cmd)
    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')    
    
    # Build square wave of 3 cycle for segment 2
    segnum = 2
    seglen = 1024
    ncycles2 =3
    cyclelen =seglen/ncycles2
    waves = [ None for _ in range(3)]
    # square wave:
    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    waves[0] = y.astype(wpt_type)
    # Download square wave to channel 1
    wav = waves[0]
#     print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))    
    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
    # Select the segment
    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000         
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}" after writing binary values'.format(resp))
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
    inst.send_scpi_cmd(cmd)
    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')    
    
       

    # Build triangle wave of 4 cycle for segment 3
    segnum = 3
    seglen = 1024
    ncycles3 = 4
    cyclelen = (seglen/ncycles3)
    waves = [ None for _ in range(3)]
    # triangle wave:
    x = np.linspace(start=0, stop=2 * np.pi * ncycles3, num=seglen, endpoint=False)
    y = np.sin(x)
    y = np.arcsin(y)* 2 * half_dac / np.pi + half_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    waves[0] = y.astype(wpt_type)
    if dac_mode == 16:
        waves[0] = y.astype(np.uint16)
    else:
        waves[0] = y.astype(np.uint8)

#     plt.plot(x,y)
#     print("************* WAVEFORM*****************")
    # Download triangle wave to channel 1
    wav = waves[0]
#     print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))    
    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
    # Select the segment
    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}" after writing binary values'.format(resp))
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
    inst.send_scpi_cmd(cmd)
    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')    
    

    # Build DC wave of 4 cycle for segment 4 
    segnum = 4
    seglen = 1024
    ncycles = 2
    max_dac = 65535
    half_dac = max_dac // 2.0
    waves = [None for _ in range(3)]
    # Create DC wave
    for i in range(1):
        cyclelen = seglen / ncycles
        x = np.linspace(1,1024,1024) 
        y = max_dac *x/x
        y = np.round(y)
        y = np.clip(y, 0, max_dac)
        if dac_mode == 16:
            waves[i] = y.astype(np.uint16)
        else:
            waves[i] = y.astype(np.uint8)
    # Download segments
    wav = waves[0]
#     print('Download wave to segment {} of channel {}'.format(segnum,channel_select[in_channelselect]))    
    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))    
    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
    # Select the segment
    inst.send_scpi_cmd(':TRAC:SEL {}'.format(segnum))
    # Increase the timeout before writing binary-data:
    inst.timeout = 30000    
    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)        
    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)
    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{}" after writing binary values'.format(resp))
    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {}'.format(segnum)
    inst.send_scpi_cmd(cmd)
    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')    

    
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
    inst.send_scpi_cmd(':INIT:CONT OFF')
    inst.send_scpi_cmd(':TRIG:SOUR:ENAB CPU')
    inst.send_scpi_cmd(':TRIG:STAT ON')

    
    
    seq_num = 3
    seq_loop = 2
    
    for i in range(int(seq_num)):
        
        # Create Task table
    
        inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format((3*i)+1))
        inst.send_scpi_cmd(':TASK:COMP:ENAB CPU') 
        inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(1))                  # play segment 2 SQP
        inst.send_scpi_cmd(':TASK:COMP:TYPE STARt')                          # start of sequence
        inst.send_scpi_cmd(':TASK:COMP:SEQ {}'.format(seq_loop))                         # sequence loop = FRAME_NMB
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format((3*i)+2))                 # the next task is 6
#         print('Task no {} is created successfully'.format((3*i)+1))

        inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format((3*i)+2))
        inst.send_scpi_cmd(':TASK:COMP:ENAB CPU') 
        inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(2))                  # play segment 2 SQP
        inst.send_scpi_cmd(':TASK:COMP:TYPE SEQ')                          # start of sequence
        inst.send_scpi_cmd(':TASK:COMP:SEQ {}'.format(seq_loop))                         # sequence loop = FRAME_NMB
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format((3*i)+3))                 # the next task is 6
#         print('Task no {} is created successfully'.format((3*i)+2))
        
        
        inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format((3*i)+3))
        inst.send_scpi_cmd(':TASK:COMP:ENAB CPU')
        inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(3))                  # play segment 5 DC
        inst.send_scpi_cmd(':TASK:COMP:TYPE END')                            # in sequence
        inst.send_scpi_cmd(':TASK:COMP:NEXT1 {0}'.format((3*i)+4))                 # next1 task is 7
#         print('Task no {} is created successfully'.format((3*i)+3))
    
        
    inst.send_scpi_cmd(':TASK:COMP:SEL {}'.format((3*seq_num)+1))
    inst.send_scpi_cmd(':TASK:COMP:ENAB CPU')
    inst.send_scpi_cmd(':TASK:COMP:SEGM {0}'.format(4))                  # play segment 5 DC
#     print('Task no {} is created successfully'.format((3*seq_num)+1))

        
    inst.send_scpi_cmd(':TASK:COMP:WRIT')
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channel_select[in_channelselect]))
    inst.send_scpi_cmd(':FUNC:MODE TASK')
    inst.send_scpi_cmd(':OUTP ON')


    scope.write(':CHANnel{}:PROBe 1.0'.format(channel_select[in_channelselect]))
    scope.write(':CHANnel{}:SCALe 200E-3'.format(channel_select[in_channelselect]))
    scope.write(':CHANnel{}:OFFSet 0.0'.format(channel_select[in_channelselect]))
    scope.write(':TIMebase:SCALe 1E-6')
    scope.write(':TRIGger:SWEep TRIGgered')
    scope.write(':TRIGger:MODE EDGE')
    scope.write(':TRIGger:EDGE:SOURce CHANnel{}'.format(channel_select[in_channelselect]))
    scope.write(':TRIGger:LEVel CHANnel{},120e-3'.format(channel_select[in_channelselect]))
    time.sleep(2)
    measu_freq = []
    peak_volt = []
    q = 0

    for i in range(((seq_num*3)*seq_loop)+1):
        time.sleep(1)
        inst.send_scpi_cmd('*TRG')
        time.sleep(3)
        scope.write(':MEASure:FREQuency CHANnel{}'.format(channel_select[in_channelselect]))
        scope.write(':MEASure:RESults?')
        time.sleep(1)
        result = scope.read()
        meas_freq = float(result.split(',')[2])/1e6
#         print(meas_freq)
        measu_freq.append(meas_freq)
        time.sleep(1)
        scope.write(':MEASure:VPP CHANnel{}'.format(channel_select[in_channelselect]))
        time.sleep(1)
        scope.write(':MEASure:RESults?')
        result = scope.read()
        peak_peak_volt= float(result.split(',')[1])
#         print(peak_peak_volt)
        peak_volt.append(peak_peak_volt)


    scope.write(':CHANnel{}:DISPlay OFF'.format(channel_select[in_channelselect]))
    
    for i in range(seq_num*seq_loop):    
        if abs(measu_freq[(i*3)+0]-1.95)<0.05 and abs(measu_freq[(i*3)+1]-2.92)<0.05 and abs(measu_freq[(i*3)+2]-3.90)<0.05:
            if 0.50<float(peak_volt[(i*3)+0])<0.62 and 0.70<float(peak_volt[(i*3)+1])<0.83 and 0.50<float(peak_volt[(i*3)+2])<0.62:
                q = 0
            else:
                q = 1
                break
        else:
            q = 2
            break
            

       
    if q==0 and measu_freq[(seq_num*seq_loop*3)]==9.99999e+31:
        print('-->Test  Pass for channel {0} \n'.format(channel_select[in_channelselect]))
    elif q==1:
        print('-->Test  Fail for channel {0} due to peak value \n'.format(channel_select[in_channelselect]))
    else:
        print('-->Test  Fail for channel {0} due to frequency \n'.format(channel_select[in_channelselect]))
        
        

#     for i in range(4):
#         scope.write(':CHANnel{}:DISPlay OFF'.format(i+1)) 
#         scope.write(':MEASure:CLEar')
#         # Clear the display for next operation

    

print('***Test Completed for all the channels***')
# close proteus connection
inst.close_instrument()
disconnect()

# Close the connection to the instrument
scope.close()
resourceManager.close()

    

Test start for channel no. 1
-->Test  Pass for channel 1 

Test start for channel no. 2
-->Test  Pass for channel 2 

Test start for channel no. 3
-->Test  Pass for channel 3 

Test start for channel no. 4


In [ ]:
#disconnect  
dis = input("Press enter to disconnect")
disconnect()
print("Device is Disconnected")